### Задача 1.

Используя данные из таблицы `user_actions`, рассчитайте среднее число заказов всех пользователей нашего сервиса.

Для этого сначала в подзапросе посчитайте, сколько заказов сделал каждый пользователь, а затем обратитесь к результату подзапроса в блоке `FROM` и уже в основном запросе усредните количество заказов по всем пользователям.

Полученное среднее число заказов всех пользователей округлите до двух знаков после запятой. Колонку с этим значением назовите `orders_avg`.

Поле в результирующей таблице: `orders_avg`

```SQL
SELECT ROUND(AVG(orders_count), 2) AS orders_avg
FROM (
    SELECT user_id, COUNT(order_id) AS orders_count
    FROM user_actions
    WHERE action != 'cancel_order'
    GROUP BY user_id
) AS subquery
```

### Задача 2.

Повторите запрос из предыдущего задания, но теперь вместо подзапроса используйте оператор `WITH` и табличное выражение.

Условия задачи те же: используя данные из таблицы `user_actions`, рассчитайте среднее число заказов всех пользователей.

Полученное среднее число заказов округлите до двух знаков после запятой. Колонку с этим значением назовите `orders_avg`.

Поле в результирующей таблице: `orders_avg`

```SQL
WITH 
subquery AS (
    SELECT user_id, COUNT(order_id) AS orders_count
    FROM user_actions
    WHERE action != 'cancel_order'
    GROUP BY user_id
)

SELECT ROUND(AVG(orders_count), 2) AS orders_avg
FROM subquery
```

### Задача 3.

Выведите из таблицы `products` информацию о всех товарах кроме самого дешёвого.

Результат отсортируйте по убыванию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`

```SQL
SELECT product_id, name, price
FROM products
WHERE price != (SELECT MIN(price) FROM products)
ORDER BY product_id DESC
```

### Задача 4.

Выведите информацию о товарах в таблице `products`, цена на которые превышает среднюю цену всех товаров на 20 рублей и более. Результат отсортируйте по убыванию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`

```SQL
SELECT product_id, name, price
FROM products
WHERE price > (SELECT AVG(price) FROM products) + 20
ORDER BY product_id DESC
```

### Задача 5.

Посчитайте количество уникальных клиентов в таблице `user_actions`, сделавших за последнюю неделю хотя бы один заказ.

Полученную колонку с числом клиентов назовите `users_count`. В качестве текущей даты, от которой откладывать неделю, используйте последнюю дату в той же таблице `user_actions`.

Поле в результирующей таблице: `users_count`

```SQL
SELECT CONUT(DISTINCT user_id) AS users_count
FROM   user_actions
WHERE  action = 'create_order'
   and time >= (SELECT MAX(time) FROM user_actions) - INTERVAL '7 day'
```

### Задача 6.

С помощью функции `AGE` и агрегирующей функции снова определите возраст самого молодого курьера мужского пола в таблице couriers, но в этот раз при расчётах в качестве первой даты используйте последнюю дату из таблицы `courier_actions`.

Чтобы получить именно дату, перед применением функции `AGE` переведите последнюю дату из таблицы `courier_actions` в формат `DATE`, как мы делали в этом задании.

Возраст курьера измерьте количеством лет, месяцев и дней и переведите его в тип `VARCHAR`. Полученную колонку со значением возраста назовите `min_age`.

Поле в результирующей таблице: `min_age`

```SQL
SELECT
  AGE(
    (
      SELECT
        MAX(time)
      FROM
        courier_actions
    )::DATE,
    birth_date
  )::VARCHAR AS min_age
FROM
  couriers
WHERE
  sex = 'male'
ORDER BY
  min_age
limit
  1
```

### Задача 7.

Из таблицы `user_actions` с помощью подзапроса или табличного выражения отберите все заказы, которые не были отменены пользователями.

Выведите колонку с `id` этих заказов. Результат запроса отсортируйте по возрастанию `id` заказа.

Добавьте в запрос оператор `LIMIT` и выведите только первые 1000 строк результирующей таблицы.

Поле в результирующей таблице: `order_id`

```SQL
SELECT
    DISTINCT order_id
FROM
    user_actions
WHERE
    order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
ORDER BY
    order_id
LIMIT 1000
```

### Задача 8.

Используя данные из таблицы `user_actions`, рассчитайте, сколько заказов сделал каждый пользователь и отразите это в столбце `orders_count`.

В отдельном столбце `orders_avg` напротив каждого пользователя укажите среднее число заказов всех пользователей, округлив его до двух знаков после запятой.

Также для каждого пользователя посчитайте отклонение числа заказов от среднего значения. Отклонение считайте так: число заказов «минус» округлённое среднее значение. Колонку с отклонением назовите `orders_diff`.

Результат отсортируйте по возрастанию `id` пользователя. Добавьте в запрос оператор `LIMIT` и выведите только первые 1000 строк результирующей таблицы.

Поля в результирующей таблице: `user_id`, `orders_count`, `orders_avg`, `orders_diff`

```SQL
SELECT
    user_id,
    COUNT(DISTINCT order_id) AS orders_count,
    ROUND((SELECT COUNT(DISTINCT order_id) /
    COUNT(DISTINCT user_id)::DECIMAL FROM user_actions), 2) AS orders_avg,
    COUNT(DISTINCT order_id) - ROUND((SELECT COUNT(DISTINCT order_id) /
    COUNT(DISTINCT user_id)::DECIMAL FROM user_actions), 2) AS orders_diff
FROM
    user_actions
GROUP BY 
    user_id
ORDER BY 
    user_id
LIMIT 1000
```